<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up" data-toc-modified-id="Set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up</a></span></li><li><span><a href="#Prelimanary-Analysis" data-toc-modified-id="Prelimanary-Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prelimanary Analysis</a></span></li><li><span><a href="#Bushfire" data-toc-modified-id="Bushfire-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bushfire</a></span></li><li><span><a href="#Appendix" data-toc-modified-id="Appendix-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Appendix</a></span><ul class="toc-item"><li><span><a href="#Person" data-toc-modified-id="Person-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Person</a></span></li><li><span><a href="#Spearman" data-toc-modified-id="Spearman-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Spearman</a></span></li><li><span><a href="#Kendall" data-toc-modified-id="Kendall-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Kendall</a></span></li></ul></li></ul></div>

# Set up

In [1]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau

import matplotlib.pyplot as plt

import numpy as np
import os
import pandas as pd
import json
import re, datetime
from geopy.geocoders import Nominatim

# Prelimanary Analysis

input = one industry data folder
output = print a pandas table: row = a company, column = time


the steps of processing progress:
1. 

In [10]:
CFP = ['ROA1', 'ROA2', 'Operating Margin', 'Net Margin', 'Return On Equity', 
           'Current Ratio', 'Quick ratio', 'ROC', 'D/E ratio', 'EPS']

def corr_ana(industry:str, climate:str, cfp_diff:int, climate_diff:int):
    """
    :industry = one of the industry name in our list
    :climate = can be temperature or bushfire data
    #:cfp = the type of CFP
    :cfp_diff = the time difference of the cfp data
    :climate_diff = the time differecne of the climate data
    output = a pandas framework 
    """
    
    # read the data of industry
    cfp_data, state_data = read_industry(industry)
    
    # read the climate data
    climate_data = read_climate(climate)
    
    # process data
    processed_data = process_data(cfp_data, state_data, climate_data, climate, cfp_diff, climate_diff)
    
    # compute the p-value
    result = compute_p(processed_data, industry, climate, cfp_diff, climate_diff)
    
    return result
    
    
def read_industry(industry:str):
    """
    :industry = one of the industry list
    output = cfp_data, state_data
    """
    root_path = os.path.abspath(os.path.dirname(os.getcwd()))
    data_path = os.path.join(root_path, 'data')
    inds_path = os.path.join(data_path, industry)

    finc_data = []
    for file in os.listdir(inds_path):
#         print(file)
        if file.endswith('.json'):
            file_path = os.path.join(inds_path, file)
            with open(file_path, 'r') as file:
                finc_data.append(json.load(file))

    cfp_data = {}
    state_data = {}
    for i in finc_data:
        for k, v in i.items():
            try:
                if i[k]['state']:
                    cfp_data[k] = {}
                    state_data[k] = i[k]['state']
                    for cfp in CFP:
                        cfp_data[k][cfp] = pd.DataFrame.from_dict(i[k][cfp]) 
            except:
                pass
            
    return cfp_data, state_data

def read_climate(climate:str):
    """
    :climate = 'bushfire' or 'temp'
    """
    
    assert climate == 'bushfire' or climate == 'temp', "plz type 'bushfire' or 'temp'"
    
    root_path = os.path.abspath(os.path.dirname(os.getcwd()))
    data_path = os.path.join(root_path, 'data')
    
    if climate == 'bushfire':
        climate_path = os.path.join(data_path, 'climate')
        file_name = 'Wildfire_data.csv'
        file_path = os.path.join(climate_path, file_name)
        climate_data = pd.read_csv(file_path)
    else:
        climate_path = os.path.join(data_path, 'climate')
        climate_path = os.path.join(climate_path, 'US_temperature_data')
        file_name = 'original_temp_data.csv'
        file_path = os.path.join(climate_path, file_name)
        climate_data = pd.read_csv(file_path)
        
    return climate_data

def process_data(cfp_data, state_data, climate_data, climate:str, cfp_diff:int, climate_diff:int):
    
    assert climate == 'bushfire' or climate == 'temp', "plz type 'bushfire' or 'temp'"
    
    if climate == 'bushfire':
        # get the bushfire and cfp data pair to feed to correlation analysis
        pairs = process_bushfire_pairs(cfp_data, state_data, climate_data, cfp_diff, climate_diff)
    else:
        # get the temperature and cfp data pair to feed
        pairs = process_temp_pairs(cfp_data, state_data, climate_data, cfp_diff, climate_diff)
        
    return pairs

def process_temp_pairs(cfp_data, state_data, temp_data, cfp_diff:int, climate_diff:int):
    """
    :cfp_data = 
    :state_data = 
    output = a nested dict, the key is the company name, and the value is a dictionary with the four keys: 
                "cfp_data", "climate_data", "cfp_year", "climate_year", and "state", 
                the "cfp_data" and "climate_data" will feed to correlation analysis
    """
    result = {}
    
    # shape data as year, cfp, temp
    for comp in cfp_data.keys():
        # init the values
        result[comp] = {}
        state = state_data[comp]
        state_temp = temp_data.loc[temp_data['State'] == state] # get state temp date
        state_temp = state_temp.set_index("Date") # set Date as row index  
        
        for c in CFP:
            result[comp][c] = {}
            cfp = []
            temp = []
            cfp_time = []
            temp_time = []
            cfp_time = []
            # get cfp timeline 
            for i in cfp_data[comp][c].iloc[:,range(1, cfp_data[comp][c].shape[1])]:
                try:
                    i = i.split('/')
                    cfp_time.append(datetime.date(int(i[-1]), int(i[0]), int(i[1])))
                except:
                    pass        
        
            # get temp timeline
            state_temp_time = []
            for i in state_temp.index.values:
                i = i.split('/')
                state_temp_time.append(datetime.date(int(i[0]), int(i[1]), int(i[2])))     
            
            # get the temp avg
            temp_curr_avg = []
            temp_past_avg = []
            cfp_index = []
            # for each interval(elementc) in roa, we need to find the corresponding temp difference
            # the corresponding temp difference = the current avg. temp - the avg. temp over past n years in the same interval
            for time in range(len(cfp_time)-1): # don't check the last element of cfp_time    
                now = cfp_time[time]
                now_year = cfp_time[time].year
                now_month = cfp_time[time].month
                now_day = cfp_time[time].day
                try: # use exisiting data to get the interval
                    past = cfp_time[time + 1]
                    past_year = cfp_time[time + 1].year
                    past_month = cfp_time[time + 1].month
                    past_day = cfp_time[time + 1].day
                except: # assume the past year for the last element
                    # to prevent nan value situation
                    past_year = now_year - 1
                    past_month = now_month
                    past_day = now_day
                    past = datetime.date(past_year, past_month, past_day)
                temp_sum = 0
                temp_count = 0
                # get temp avg. of current year 
                for temp in range(len(state_temp_time)):
                    if past <= state_temp_time[temp] < now:
                        year = str(state_temp_time[temp].year)
                        month = str(state_temp_time[temp].month)
                        day = str(state_temp_time[temp].day)
                        timestamp = year + '/' + month + '/' + day
                        temp_sum += state_temp.loc[timestamp].values[0]
                        temp_count += 1       
                if temp_count == 0: # can't find corresponding data points
                    continue
                temp_curr_avg.append(temp_sum/temp_count)
                temp_sum = 0
                temp_count = 0
                # get temp avg. of the past n years
                for n in range(1, climate_diff+1):
                    now_year -= 1
                    past_year -= 1
                    now = datetime.date(now_year, now_month, now_day)
                    past = datetime.date(past_year, past_month, past_day)
                    for temp in range(len(state_temp_time)):
                        if past <= state_temp_time[temp] < now:
                            year = str(state_temp_time[temp].year)
                            month = str(state_temp_time[temp].month)
                            day = str(state_temp_time[temp].day)
                            timestamp = year + '/' + month + '/' + day
                            temp_sum += state_temp.loc[timestamp].values[0]
                            temp_count += 1
                if climate_diff != 0:
                    temp_past_avg.append(temp_sum/temp_count)
                else:
                    temp_past_avg.append(0)
                cfp_index.append(time)
                temp_time.append(cfp_time[time])
            columns = list(cfp_data[comp][c].columns[1:])
            # get cfp data
            for i in range(len(columns)):
                if i in cfp_index:
                    if i + cfp_diff >= len(columns):
                        # i = current cfp data point
                        # i + cfp_diff = the farest data cfp data point
                        # if count + cfp_diff is out of range, terminate computing the cfp moving avg.,
                        # otherwise the array can provide sufficient data points 
                        break
                    try:
                        cfp_curr = float(cfp_data[comp][c][columns[i]].values[0])
                    except:
                        cfp_curr = 0
                    cfp_past = 0
                    for n in range(1, cfp_diff+1):
                        cfp_past += float(cfp_data[comp][c][columns[i+n]].values[0])
                    if cfp_diff != 0:
                        cfp_avg = cfp_past/cfp_diff
                    else:
                        cfp_avg = 0
                    # get the cfp difference
                    cfp.append(cfp_curr - cfp_avg)
                    cfp_time.append(columns[i])
            # get temp difference
            temp = [a - b for a, b in zip(temp_curr_avg, temp_past_avg)]
        
            result[comp][c]["cfp_data"] = cfp
            result[comp][c]["climate_data"] = temp
            result[comp][c]["cfp_year"] = cfp_time
            result[comp][c]["climate_year"] = temp_time
            result[comp][c]['state'] = state
    return result   

def process_bushfire_pairs(cfp_data, state_data, bushfire_data, cfp_diff:int, climate_diff:int):
    """
    :cfp_data = 
    :state_data = 
    output = a nested dict, the key is the company name, and the value is a dictionary with the four keys: 
                "cfp_data", "climate_data", "cfp_year", "climate_year", and "state", 
                the "cfp_data" and "climate_data" will feed to correlation analysis
    """
    
    result = {}
    
    # shape data as year, cfp, bushfire
    for comp in cfp_data.keys():
        # init the values
        result[comp] = {}
        
        for c in CFP:
            result[comp][c] = {}
            cfp = []
            bushfire = []
            time = []
            state = state_data[comp]
            result[comp]['state'] = state

            for i in cfp_data[comp][c].iloc[:,range(1, cfp_data[comp][c].shape[1])]:
                # get the year of cfp dta
                year = i.split('/')[-1]
                try:
                    # get the corresponding year data of bushfire
                    bushfire_value = bushfire_data.loc[bushfire_data['Year'] == int(year)]['Acres']
                    # shape the string to int
                    bushfire_value = int(bushfire_value.values[0].replace(',', ''))
                    if bushfire_value:
                        cfp_value = cfp_data[comp][c][i][0]
                        cfp_value = float(cfp_value)
                        if cfp_value and cfp_value is not np.isnan(cfp_value):
                            cfp.append(cfp_value)
                            bushfire.append(bushfire_value)
                            time.append(i)
                except:
                    pass

            # compute the difference of cfp
            result[comp][c]["cfp_data"] = compute_diff(comp, cfp, cfp_diff, t=c)
            # compute the difference of cfp
            result[comp][c]["climate_data"] = compute_diff(comp, bushfire, climate_diff, t='bushfire')
            # reshape the time
            if cfp_diff == 0:
                result[comp][c]["cfp_year"] = time
            else:
                result[comp][c]["cfp_year"] = time[:-cfp_diff]
            if climate_diff == 0:
                result[comp][c]["climate_year"] = time
            else:
                result[comp][c]["climate_year"] = time[:-climate_diff]
    
    return result
        
def compute_diff(comp:str, data:list, time_diff:int, t:str):
    """
    output = processed data
    """
    
    result = []
    for i in range(len(data)-time_diff):
        current = data[i]
        # python follows left close and right open
        if time_diff == 0:
            result.append(current)
        else:
            moving_avg = sum(data[i+1:i+time_diff+1])/time_diff
            # remain 5 deciamls
            diff = round(current - moving_avg, 5)
            result.append(diff)
    if len(result) <= 5:
        print(comp, 'only has', len(result), 'data points after computing', 
              time_diff, 'years moving avg over', t, 'data')
        
    return result


def compute_p(data:dict, industry:str, climate:str, cfp_diff:int, climate_diff:int):
    """
    :data = a nested dictionary from process_data function
    """
    def cat_float(a:float, b:float) -> str:
        a = str(round(a, 3))
        b = str(round(b, 3))
        return a + '(' + b + ')'

    columns = pd.MultiIndex.from_product([['ROA1', 'ROA2', 'Operating Margin', 'Net Margin', 'Return On Equity', 
           'Current Ratio', 'Quick ratio', 'ROC', 'D/E ratio', 'EPS'], ['Pearson', 'Spearman', 'Kendall']],
                                        names = ["CFP", "Correlation"])
    index = pd.Index(data.keys(), name = "Company:")
    df = pd.DataFrame(columns = columns, index = index)
    
    for comp in data.keys():
        for c in CFP:
            print(comp,c)
            
            result = []
            l1 = data[comp][c]["cfp_data"]
            l2 = data[comp][c]["climate_data"]
            print(l1, l2)
            if len(l1) != l2:
                min_len = min(len(l1), len(l2))
                l1 = l1[:min_len]
                l2 = l2[:min_len]
            if len(l1) > 2:
                # Pearson correlation
                pearson = pearsonr(l1, l2)[0]
                p_p_value = pearsonr(l1, l2)[1]
#                 print(cat_float(pearson, p_p_value))
#                 df.loc[comp][c]["Pearson"] = cat_float(pearson, p_p_value)
                result.append(cat_float(pearson, p_p_value))
                # Spearman correlation
                spearman = spearmanr(l1, l2)[0]
                s_p_value = spearmanr(l1, l2)[1]
#                 print(cat_float(spearman, s_p_value))
#                 df.loc[comp][c]["Spearman"] = cat_float(spearman, s_p_value)
                result.append(cat_float(spearman, s_p_value))
                # Kendall correlation
                tau, k_p_value = kendalltau(l1, l2)
#                 print(cat_float(tau, k_p_value), '\n')
#                 df.loc[comp][c]["Kendall"] = cat_float(tau, k_p_value)
                result.append(cat_float(tau, k_p_value))
                df.loc[comp, c] = result
        
    title = 'The correlation result between '
    title += industry + '(' + str(cfp_diff) + ' years moving avg.)'
    title += ' and ' + climate + '(' + str(climate_diff) + ' years moving avg.).csv'
    df.to_csv(title)
    
    return df.style.set_caption(title)

# Bushfire

In [13]:
corr_ana("agriculture", 'bushfire', 0, 0)

YTEN only has 2 data points after computing 0 years moving avg over D/E ratio data
YTEN only has 2 data points after computing 0 years moving avg over bushfire data
CTA-PB only has 0 data points after computing 0 years moving avg over ROA1 data
CTA-PB only has 0 data points after computing 0 years moving avg over bushfire data
CTA-PB only has 0 data points after computing 0 years moving avg over ROA2 data
CTA-PB only has 0 data points after computing 0 years moving avg over bushfire data
CTA-PB only has 0 data points after computing 0 years moving avg over Operating Margin data
CTA-PB only has 0 data points after computing 0 years moving avg over bushfire data
CTA-PB only has 0 data points after computing 0 years moving avg over Net Margin data
CTA-PB only has 0 data points after computing 0 years moving avg over bushfire data
CTA-PB only has 0 data points after computing 0 years moving avg over Return On Equity data
CTA-PB only has 0 data points after computing 0 years moving avg over

ValueError: array must not contain infs or NaNs

In [26]:
corr_ana("energy", 'bushfire', 'ROA1', 0, 0)

,Pearson,Spearman,Kendall
PTEN,-0.018(0.928),-0.081(0.675),-0.02(0.896)
ICD,-0.108(0.783),0.033(0.932),0.056(0.919)
HP,0.069(0.687),0.15(0.382),0.114(0.327)


In [93]:
corr_ana("travel", 'bushfire', 'ROA2', 0, 5)

NCLH only has 0 data points after computing 0 years moving avg over cfp data
NCLH only has 0 data points after computing 5 years moving avg over bushfire data
LIND only has 4 data points after computing 5 years moving avg over bushfire data


/Users/dnhb/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:329: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Pearson,Spearman,Kendall
TRIP,-0.203(0.7),-0.086(0.872),-0.2(0.719)
TZOO,0.211(0.417),0.211(0.417),0.132(0.49)
NCLH,nan,nan,nan
LIND,-0.446(0.554),-0.8(0.2),-0.667(0.333)
TNL,0.222(0.446),0.279(0.334),0.209(0.331)
MKGI,0.073(0.863),-0.024(0.955),0.0(1.0)
EXPE,0.148(0.614),-0.09(0.759),-0.055(0.83)


In [32]:
corr_ana("agriculture", 'bushfire', 'ROA1', 0, 3)

CTA-PB only has 0 data points after computing 0 years moving avg over cfp data
CTA-PB only has 0 data points after computing 3 years moving avg over bushfire data
CTVA only has 0 data points after computing 0 years moving avg over cfp data
CTVA only has 0 data points after computing 3 years moving avg over bushfire data
CTA-PA only has 0 data points after computing 0 years moving avg over cfp data
CTA-PA only has 0 data points after computing 3 years moving avg over bushfire data


/Users/dnhb/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:329: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Pearson,Spearman,Kendall
RKDA,0.347(0.501),0.257(0.623),0.2(0.719)
YTEN,0.139(0.636),0.103(0.725),0.121(0.591)
AVD,0.149(0.415),0.098(0.595),0.073(0.573)
CTA-PB,nan,nan,nan
IPI,-0.394(0.206),-0.119(0.713),-0.03(0.947)
MGPI,0.192(0.31),0.125(0.511),0.067(0.62)
SMG,-0.119(0.546),-0.028(0.886),-0.011(0.953)
FMC,0.174(0.34),0.113(0.538),0.097(0.449)
MBII,-0.037(0.937),0.0(1.0),0.048(1.0)
CTVA,nan,nan,nan


In [59]:
corr_ana("travel", 'bushfire', 'ROA1', 0, 0)

NCLH only has 0 data points after computing 0 years moving avg over cfp data
NCLH only has 0 data points after computing 0 years moving avg over bushfire data


/Users/dnhb/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:329: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Pearson,Spearman,Kendall
TRIP,-0.442(0.15),-0.462(0.131),-0.364(0.116)
TZOO,-0.103(0.64),-0.179(0.414),-0.115(0.464)
NCLH,nan,nan,nan
LIND,-0.314(0.378),-0.188(0.603),-0.156(0.601)
TNL,0.01(0.968),-0.081(0.734),-0.084(0.631)
MKGI,-0.255(0.378),-0.16(0.584),-0.121(0.591)
EXPE,0.061(0.8),0.045(0.85),0.021(0.924)


# Appendix

## Person

$$
r=\frac{\sum\left(x-m_{x}\right)\left(y-m_{y}\right)}{\sqrt{\sum\left(x-m_{x}\right)^{2} \sum\left(y-m_{y}\right)^{2}}}
$$

REF: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html

## Spearman

$$
\rho=1-\frac{6 \sum d_{i}^{2}}{n\left(n^{2}-1\right)}
$$

$\rho$ 	=	Spearman's rank correlation coefficient  
$d_i$ = difference between the two ranks of each observation  
$n$ = number of observations

REF1: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html  
REF2: https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient

## Kendall

$$
\tau=\frac{(\text { number of concordant pairs })-(\text { number of discordant pairs })}{\left(\begin{array}{c}
n \\
2
\end{array}\right)}
$$

REF: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html?highlight=kendall#scipy.stats.kendalltau